In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import torch
import json

In [14]:
prediction_result = torch.load('work_dirs/ped/vfnet_r101/init/eval/prediction_epoch_21.pth') # 21 0.785
prediction_result_opt = [it for it in prediction_result if it['score'] >= 0]

temp_json = 'work_dirs/ped/gfl/init/eval/temp.json'
with open(temp_json, 'w') as f:
    f.write(json.dumps(prediction_result_opt))
    f.flush()
    

#####################################################
gt_json = "data/ped/annotations/dhd_traffic_val.json"
res_file = temp_json

cocoGt = COCO(gt_json)
cocoDt = cocoGt.loadRes(res_file) 
coco_eva = COCOeval(cocoGt, cocoDt, 'bbox')

img_ids_in_cat = set(cocoGt.catToImgs[1])
coco_eva.params.catIds = [1] #person id : 1
# coco_eva.params.imgIds = list(img_ids_in_cat)

coco_eva.evaluate()
coco_eva.accumulate()
out = coco_eva.summarize()

loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.41s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.48s).
Accumulating evaluation results...
DONE (t=0.93s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.438
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.777
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.441
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.237
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.475
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.620
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.232
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.536
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

In [15]:
#######

import json
import pandas as pd

to_path = '/home/lqurszh/Documents/Data/Ped/results/merge/vfn_ep22.csv'

with open(res_file) as f:
    anns = json.load(f)
ImgIds = set( [ann['image_id'] for ann in anns] )
print("output image size: {}".format(len(ImgIds)))

data_dict = {
    "image_id": [],
    "score": [],
    "x": [],
    "y": [],
    "width": [],
    "height": [],
}

for res in anns:
    if res['category_id'] == 1:
        x,y,w,h = list(map(int, res['bbox']))
        data_dict["image_id"].append(res['image_id'])
        data_dict["score"].append(res['score'])
        data_dict["x"].append(x)
        data_dict["y"].append(y)
        data_dict["width"].append(w)
        data_dict["height"].append(h)
print('pedestrain detection size: ', len(data_dict['image_id']))
df = pd.DataFrame(data_dict)
df.to_csv(to_path, index=False)
print("save output as {}".format(to_path))


output image size: 4997
pedestrain detection size:  61606
save output as /home/lqurszh/Documents/Data/Ped/results/merge/vfn_ep22.csv
